In [1]:
from sklearn import preprocessing
import pandas as pd
import numpy as np
from math import pi, sin, cos
from datetime import datetime
from holidays_es import get_provinces, Province

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

2022-08-30 17:45:29.644235: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-30 17:45:29.644284: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Importing data

In [2]:
df = pd.read_json("../raw_data/201808.json", encoding_errors="ignore", lines=True)
weather = pd.read_csv("../raw_data/weather/weather_cleaned.csv")
df.head()

,_id,stations
0,2018-08-01T00:58:32.872907,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
1,2018-08-01T01:58:33.570038,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
2,2018-08-01T02:58:36.692852,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
3,2018-08-01T03:58:37.185141,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."
4,2018-08-01T04:58:40.413655,"[{'activate': 1, 'name': 'Puerta del Sol A', '..."


## Adding columns

In [3]:
%%time
df["time"] = "time"
df["date"] = "date"
for i in range(len(df['_id'])):
    item=df['_id'][i]
    groups= item.split("T")
    date = groups[0]
    raw_time = groups[1].split(":")
    time =':'.join(raw_time[:2])
    df["time"][i] = time
    df["date"][i] = date
df.head(3)

CPU times: user 342 ms, sys: 35.2 ms, total: 377 ms
Wall time: 351 ms


,_id,stations,time,date
0,2018-08-01T00:58:32.872907,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",00:58,2018-08-01
1,2018-08-01T01:58:33.570038,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",01:58,2018-08-01
2,2018-08-01T02:58:36.692852,"[{'activate': 1, 'name': 'Puerta del Sol A', '...",02:58,2018-08-01


In [4]:
df['time'] = df.apply(lambda x:datetime.strptime(x['time'], '%H:%M').hour, axis=1)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')


In [5]:
holidays=[]
for year in [2018,2019,2020,2021]:
    temp = Province(name="madrid", year=year).holidays()
    for day in range(len(temp['national_holidays'])):
        holidays.append(temp['national_holidays'][day])
    for day in range(len(temp['regional_holidays'])):
        holidays.append(temp['regional_holidays'][day])
    for day in range(len(temp['local_holidays'])):
        holidays.append(temp['local_holidays'][day])

In [6]:
df_holidays=pd.DataFrame(holidays, columns=['date'])
df_holidays['date'] = pd.to_datetime(df_holidays['date'], format='%Y-%m-%d')
list_holidays=list(df_holidays['date'])
df['holidays']=df['date'].isin(list_holidays)

In [7]:
set(df.holidays)

{False, True}

# THIS TAKES A LOT TO RUN

In [28]:
len(df["stations"][172])

172

In [26]:
temp=pd.DataFrame()
for i in range(len(df)):
    for j in range(len(df["stations"][1])):
        new_dict= df['stations'][i][j]
        new_dict['id']=df['_id'][i]
        new_dict['time']=int(df['time'][i])
        new_dict['date']=df['date'][i]
        new_dict['holidays']=df['holidays'][i]
        temp = temp.append(new_dict, ignore_index=True)


KeyboardInterrupt: 

In [9]:
temp["datetime"] = df["date"].astype(str)+df["time"].astype(str)
temp

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,latitude,dock_bikes,id,time,date,holidays,datetime
0,1.0,Puerta del Sol A,0.0,2.0,24.0,11.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,11.0,2018-08-01T00:58:32.872907,0.0,2018-08-01,0.0,2018-08-010
1,1.0,Puerta del Sol A,0.0,0.0,24.0,18.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,4.0,2018-08-01T01:58:33.570038,1.0,2018-08-01,0.0,2018-08-011
2,1.0,Puerta del Sol A,0.0,2.0,24.0,13.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,10.0,2018-08-01T02:58:36.692852,2.0,2018-08-01,0.0,2018-08-012
3,1.0,Puerta del Sol A,0.0,0.0,24.0,17.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,4.0,2018-08-01T03:58:37.185141,3.0,2018-08-01,0.0,2018-08-013
4,1.0,Puerta del Sol A,0.0,0.0,24.0,22.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,0.0,2018-08-01T04:58:40.413655,4.0,2018-08-01,0.0,2018-08-014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,1.0,Puerta del Sol A,0.0,2.0,24.0,8.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,13.0,2018-08-31T19:20:59.506759,19.0,2018-08-31,0.0,2018-08-3119
751,1.0,Puerta del Sol A,0.0,1.0,24.0,6.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,16.0,2018-08-31T20:21:00.259835,20.0,2018-08-31,0.0,2018-08-3120
752,1.0,Puerta del Sol A,0.0,1.0,24.0,0.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,22.0,2018-08-31T21:20:59.808767,21.0,2018-08-31,0.0,2018-08-3121
753,1.0,Puerta del Sol A,0.0,1.0,24.0,1.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,21.0,2018-08-31T22:21:03.580436,22.0,2018-08-31,0.0,2018-08-3122


In [10]:
weather

,Unnamed: 0,feels_like,weather_main,datetime
0,0,3.08,Clear,2018-01-010
1,1,2.88,Clear,2018-01-011
2,2,2.11,Clear,2018-01-012
3,3,2.48,Clear,2018-01-013
4,4,1.69,Clear,2018-01-014
...,...,...,...,...
41488,41488,28.51,Clear,2022-08-2719
41489,41489,27.64,Clear,2022-08-2720
41490,41490,27.38,Clear,2022-08-2721
41491,41491,26.65,Clear,2022-08-2722


In [11]:
temp = temp.merge(weather, on="datetime")
temp

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,latitude,dock_bikes,id,time,date,holidays,datetime,Unnamed: 0,feels_like,weather_main
0,1.0,Puerta del Sol A,0.0,2.0,24.0,11.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,11.0,2018-08-01T00:58:32.872907,0.0,2018-08-01,0.0,2018-08-010,5197,23.19,Clear
1,1.0,Puerta del Sol A,0.0,0.0,24.0,18.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,4.0,2018-08-01T01:58:33.570038,1.0,2018-08-01,0.0,2018-08-011,5198,22.93,Clear
2,1.0,Puerta del Sol A,0.0,2.0,24.0,13.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,10.0,2018-08-01T02:58:36.692852,2.0,2018-08-01,0.0,2018-08-012,5199,21.63,Clear
3,1.0,Puerta del Sol A,0.0,0.0,24.0,17.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,4.0,2018-08-01T03:58:37.185141,3.0,2018-08-01,0.0,2018-08-013,5200,18.72,Clear
4,1.0,Puerta del Sol A,0.0,0.0,24.0,22.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,0.0,2018-08-01T04:58:40.413655,4.0,2018-08-01,0.0,2018-08-014,5201,17.94,Clear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
750,1.0,Puerta del Sol A,0.0,2.0,24.0,8.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,13.0,2018-08-31T19:20:59.506759,19.0,2018-08-31,0.0,2018-08-3119,5936,28.84,Clear
751,1.0,Puerta del Sol A,0.0,1.0,24.0,6.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,16.0,2018-08-31T20:21:00.259835,20.0,2018-08-31,0.0,2018-08-3120,5937,27.81,Clear
752,1.0,Puerta del Sol A,0.0,1.0,24.0,0.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,22.0,2018-08-31T21:20:59.808767,21.0,2018-08-31,0.0,2018-08-3121,5938,27.24,Clouds
753,1.0,Puerta del Sol A,0.0,1.0,24.0,1.0,1a,-3.7024255,0.0,Puerta del Sol n 1,40.4168961,21.0,2018-08-31T22:21:03.580436,22.0,2018-08-31,0.0,2018-08-3122,5939,26.65,Clear


In [12]:

temp['weekday'] = temp.apply(lambda x: x['date'].weekday(), axis=1)
temp['year'] = temp.apply(lambda x:x['date'].year, axis=1)
temp['month'] = temp.apply(lambda x: x['date'].month, axis=1)

In [13]:
temp.head(2)

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,...,time,date,holidays,datetime,Unnamed: 0,feels_like,weather_main,weekday,year,month
0,1.0,Puerta del Sol A,0.0,2.0,24.0,11.0,1a,-3.7024255,0.0,Puerta del Sol n 1,...,0.0,2018-08-01,0.0,2018-08-010,5197,23.19,Clear,2,2018,8
1,1.0,Puerta del Sol A,0.0,0.0,24.0,18.0,1a,-3.7024255,0.0,Puerta del Sol n 1,...,1.0,2018-08-01,0.0,2018-08-011,5198,22.93,Clear,2,2018,8


### Encoding circular time
temp['hour_sin'] = temp.apply(lambda x: sin(x['hour'] / 24.0 * 2 * pi), axis=1)
temp['hour_cos'] = temp.apply(lambda x: cos(x['hour'] / 24.0 * 2 * pi), axis=1)
temp['weekday_sin'] = temp.apply(lambda x: sin(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['weekday_cos'] = temp.apply(lambda x: cos(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['month_sin'] = temp.apply(lambda x: sin(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)
temp['month_cos'] = temp.apply(lambda x:cos(((x['month'] - 5) % 12) / 12.0 * 2 * pi), axis=1)
temp['season_sin'] = temp.apply(lambda x: sin(((x['season'] - 3) % 4) / 4.0 * 2 * pi), axis=1)
temp['season_cos'] = temp.apply(lambda x: cos(((x['season'] - 3) % 4) / 4.0 * 2 * pi), axis=1)

In [14]:
temp['hour_sin'] = temp.apply(lambda x: sin(x['time'] / 24.0 * 2 * pi), axis=1)
temp['hour_cos'] = temp.apply(lambda x: cos(x['time'] / 24.0 * 2 * pi), axis=1)
temp['weekday_sin'] = temp.apply(lambda x: sin(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['weekday_cos'] = temp.apply(lambda x: cos(x['weekday'] / 7.0 * 2 * pi), axis=1)
temp['month_sin'] = temp.apply(lambda x: sin(((x['month']- 5) % 12) / 12.0 * 2 * pi), axis=1)
temp['month_cos'] = temp.apply(lambda x:cos(((x['month'] - 5 ) % 12) / 12.0 * 2 * pi), axis=1)

In [15]:
temp.month

0      8
1      8
2      8
3      8
4      8
      ..
750    8
751    8
752    8
753    8
754    8
Name: month, Length: 755, dtype: int64

In [16]:
temp.drop(columns=["name","longitude","address","month","time","weekday","Unnamed: 0"])
temp.head(3)

,activate,name,reservations_count,light,total_bases,free_bases,number,longitude,no_available,address,...,weather_main,weekday,year,month,hour_sin,hour_cos,weekday_sin,weekday_cos,month_sin,month_cos
0,1.0,Puerta del Sol A,0.0,2.0,24.0,11.0,1a,-3.7024255,0.0,Puerta del Sol n 1,...,Clear,2,2018,8,0.000000,1.000000,0.974928,-0.222521,1.0,6.123234e-17
1,1.0,Puerta del Sol A,0.0,0.0,24.0,18.0,1a,-3.7024255,0.0,Puerta del Sol n 1,...,Clear,2,2018,8,0.258819,0.965926,0.974928,-0.222521,1.0,6.123234e-17
2,1.0,Puerta del Sol A,0.0,2.0,24.0,13.0,1a,-3.7024255,0.0,Puerta del Sol n 1,...,Clear,2,2018,8,0.500000,0.866025,0.974928,-0.222521,1.0,6.123234e-17


In [17]:
#temp.to_csv("filename.processed.csv")